In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json 
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from gensim.models import KeyedVectors
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets/tweets_DM.json
/kaggle/input/googlenews/GoogleNews-vectors-negative300.bin
/kaggle/input/glove6b100dtxt/glove.6B.100d.txt
/kaggle/input/data-id/data_identification.csv
/kaggle/input/emotion/emotion.csv


In [2]:
file=open('../input/tweets/tweets_DM.json')
data=[]
for line in file.readlines():
    data.append(json.loads(line)) 
print(type(data))

<class 'list'>


In [3]:
file.close

<function TextIOWrapper.close()>

In [4]:
data_pro=[]
for i in data:
    data_pro.append(i["_source"]['tweet']) ##get id and text and #

In [5]:
print(data_pro[0]["text"].lower())

people who post "add me on #snapchat" must be dehydrated. cuz man.... that's <lh>


In [6]:
for i in data_pro:
    i["text"]=i["text"].lower()

In [7]:
data_pro[0]

{'hashtags': ['Snapchat'],
 'tweet_id': '0x376b20',
 'text': 'people who post "add me on #snapchat" must be dehydrated. cuz man.... that\'s <lh>'}

In [8]:
data=pd.DataFrame(data_pro)
data=data.drop(columns=["hashtags"]) 
print(data)
del data_pro

         tweet_id                                               text
0        0x376b20  people who post "add me on #snapchat" must be ...
1        0x2d5350  @brianklaas as we see, trump is dangerous to #...
2        0x28b412  confident of your obedience, i write to you, k...
3        0x1cd5b0                now issa is stalking tasha 😂😂😂 <lh>
4        0x2de201  "trust is not the same as faith. a friend is s...
...           ...                                                ...
1867530  0x316b80  when you buy the last 2 tickets remaining for ...
1867531  0x29d0cb  i swear all this hard work gone pay off one da...
1867532  0x2a6a4f  @parcel2go no card left when i wasn't in so i ...
1867533  0x24faed  ah, corporate life, where you can date <lh> us...
1867534  0x34be8c             blessed to be living #sundayvibes <lh>

[1867535 rows x 2 columns]


In [9]:
file_csv_id=open(r'../input/data-id/data_identification.csv')
file_csv_em=open(r'../input/emotion/emotion.csv')

In [10]:
csv_read_data=csv.reader(file_csv_id)
csv_read_list = list(csv_read_data)
del file_csv_id

In [11]:
identity=pd.DataFrame(csv_read_list)
identity.drop([0],inplace=True)
identity.rename(columns={0:"tweet_id",1:"class"},inplace=True)

In [12]:
csv_read_data=csv.reader(file_csv_em)
csv_read_list = list(csv_read_data)
emotion=pd.DataFrame(csv_read_list)
emotion.drop([0],inplace=True)
emotion.rename(columns={0:"tweet_id",1:"emotion"},inplace=True)
del file_csv_em

In [13]:
emotion

,tweet_id,emotion
1,0x3140b1,sadness
2,0x368b73,disgust
3,0x296183,anticipation
4,0x2bd6e1,joy
5,0x2ee1dd,anticipation
...,...,...
1455559,0x38dba0,joy
1455560,0x300ea2,joy
1455561,0x360b99,fear
1455562,0x22eecf,joy


In [14]:
data=pd.merge(data,identity)

In [15]:
data

,tweet_id,text,class
0,0x376b20,"people who post ""add me on #snapchat"" must be ...",train
1,0x2d5350,"@brianklaas as we see, trump is dangerous to #...",train
2,0x28b412,"confident of your obedience, i write to you, k...",test
3,0x1cd5b0,now issa is stalking tasha 😂😂😂 <lh>,train
4,0x2de201,"""trust is not the same as faith. a friend is s...",test
...,...,...,...
1867530,0x316b80,when you buy the last 2 tickets remaining for ...,test
1867531,0x29d0cb,i swear all this hard work gone pay off one da...,test
1867532,0x2a6a4f,@parcel2go no card left when i wasn't in so i ...,test
1867533,0x24faed,"ah, corporate life, where you can date <lh> us...",train


In [16]:
max_len=len(max(data['text'],key=len))
t = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ') ##when we do tokenlize we don't want to tokenlize some mark so filt it
t.fit_on_texts(data["text"])
vocab_size = len(t.word_index) + 1
# integer encode the documents
data["text"] =t.texts_to_sequences(data["text"])
data["text"] = list(pad_sequences(data["text"], maxlen=max_len, padding='post'))

In [17]:
data["text"]

0          [59, 61, 555, 1118, 17, 13, 1193, 301, 18, 205...
1          [14258, 52, 30, 75, 202, 8, 2553, 3, 33027, 27...
2          [2982, 11, 16, 3679, 4, 942, 3, 7, 951, 15, 7,...
3          [58, 608, 8, 6616, 2449, 933, 1, 0, 0, 0, 0, 0...
4          [364, 8, 27, 2, 217, 52, 224, 5, 309, 8, 145, ...
                                 ...                        
1867530    [31, 7, 229, 2, 120, 63, 757, 6064, 9, 5, 171,...
1867531    [4, 1204, 26, 20, 259, 89, 526, 511, 132, 48, ...
1867532    [80807, 47, 1021, 326, 31, 4, 751, 10, 22, 4, ...
1867533    [2114, 4609, 32, 150, 7, 43, 837, 1, 752, 25, ...
1867534    [179, 3, 18, 424, 15283, 1, 0, 0, 0, 0, 0, 0, ...
Name: text, Length: 1867535, dtype: object

In [18]:
##data["text"]=list(np.asarray(list(data["text"].to_numpy())).astype('int32'))

In [19]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../input/glove6b100dtxt/glove.6B.100d.txt') ##This file is like googlenews to transform word to vector i chose this model is because googlenew is dim 300 to large that session will refrash(over ram)
##And this model is 100 dim 
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [20]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items(): ##Make a matric that can transform word to vecter
    try:
        embedding_vector = embeddings_index[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        pass

In [21]:
data

,tweet_id,text,class
0,0x376b20,"[59, 61, 555, 1118, 17, 13, 1193, 301, 18, 205...",train
1,0x2d5350,"[14258, 52, 30, 75, 202, 8, 2553, 3, 33027, 27...",train
2,0x28b412,"[2982, 11, 16, 3679, 4, 942, 3, 7, 951, 15, 7,...",test
3,0x1cd5b0,"[58, 608, 8, 6616, 2449, 933, 1, 0, 0, 0, 0, 0...",train
4,0x2de201,"[364, 8, 27, 2, 217, 52, 224, 5, 309, 8, 145, ...",test
...,...,...,...
1867530,0x316b80,"[31, 7, 229, 2, 120, 63, 757, 6064, 9, 5, 171,...",test
1867531,0x29d0cb,"[4, 1204, 26, 20, 259, 89, 526, 511, 132, 48, ...",test
1867532,0x2a6a4f,"[80807, 47, 1021, 326, 31, 4, 751, 10, 22, 4, ...",test
1867533,0x24faed,"[2114, 4609, 32, 150, 7, 43, 837, 1, 752, 25, ...",train


In [22]:
X_train=data[data['class']=="train"]
Final_Predict=data[data['class']=="test"]
del data
gc.collect() ## for reducing RAM

126

In [23]:
X_train=pd.merge(X_train,emotion)

In [24]:
X_train

,tweet_id,text,class,emotion
0,0x376b20,"[59, 61, 555, 1118, 17, 13, 1193, 301, 18, 205...",train,anticipation
1,0x2d5350,"[14258, 52, 30, 75, 202, 8, 2553, 3, 33027, 27...",train,sadness
2,0x1cd5b0,"[58, 608, 8, 6616, 2449, 933, 1, 0, 0, 0, 0, 0...",train,fear
3,0x1d755c,"[111557, 181667, 1871, 9, 2, 107, 60, 164, 33,...",train,joy
4,0x2c91a8,"[91, 331, 13, 182, 5956, 297151, 1, 0, 0, 0, 0...",train,anticipation
...,...,...,...,...
1455558,0x321566,"[38, 22, 95, 96926, 2, 300, 11, 20, 171, 95, 1...",train,joy
1455559,0x38959e,"[10, 140, 1070812, 442, 37, 3, 18, 258, 3, 2, ...",train,joy
1455560,0x2cbca6,"[400, 1148, 221, 429, 981, 271, 2, 3006, 25, 9...",train,joy
1455561,0x24faed,"[2114, 4609, 32, 150, 7, 43, 837, 1, 752, 25, ...",train,joy


In [25]:
X=X_train["text"]
Y=X_train["emotion"]

In [26]:
#x_train,x_test,y_train,y_test=model_selection.train_test_split(X,Y,test_size=0.3)

In [27]:
label_encoder = LabelEncoder()
label_encoder.fit(Y)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)
Y=label_encode(label_encoder, Y)
#y_train = label_encode(label_encoder, y_train)
#y_test = label_encode(label_encoder, y_test)


In [28]:
#x_train

In [29]:
X=np.asarray(list(X.to_numpy())).astype('int32')
#x_train=np.asarray(list(x_train.to_numpy())).astype('int32')
#x_test=np.asarray(list(x_test.to_numpy())).astype('int32')

In [30]:
# define model
model = Sequential() 
e = Embedding(vocab_size, 100,\ weights=[embedding_matrix], input_length=max_len, trainable=False) ## transform word to vector by embedding layer adn use matric we just made
model.add(e)
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(128,activation='relu'))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(128,activation='relu'))
model.add(Dense(8, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# checkpoint
filepath="./model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True,mode='max')
callbacks_list = [checkpoint]
# summarize the model
print(model.summary())
# fit the model
model.fit(X, Y,batch_size=64 ,epochs=25,callbacks=callbacks_list, validation_split=0.3)

SyntaxError: unexpected character after line continuation character (2729488375.py, line 3)

In [ ]:
# evaluate the model
#loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
#print('Accuracy: %f' % (accuracy*100))

In [ ]:
#model.save("./version2.h5")

In [ ]:
import csv

In [ ]:
with open('output.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for i in label_decode(label_encoder,model.predict(np.asarray(list(Final_Predict["text"].to_numpy())).astype('int32'))):
        writer.writerow([i])

In [ ]:
label_decode(label_encoder,model.predict(np.asarray(list(Final_Predict["text"].to_numpy())).astype('int32')))

In [ ]:
len(Final_Predict["text"])